# How to implement testing for Covid-19

## The stages

### Demand models

In [ ]:
def demand_test_if_experience_symptoms(states, params):
    """Demand test if person experiences symptoms.
    
    This demand model assumes that individuals request a test for a
    Covid-19 infection if they experience symptoms with a probability of 1.
    
    We select individuals who just got symptomatic by selecting the individuals where
    the countdown for getting symptoms has reached 0, is exhausted.
    
    Args:
        states (pandas.DataFrame): The states of the individuals.
        params (pandas.DataFrame): A DataFrame with parameters.
        
    Returns:
        demand_probability (numpy.ndarray, pandas.Series): An array or a series
            which contains the probability for each individual demanding a test.
    
    """
    s = pd.Series(index=states.index, data=0)
    s = s.where(states.cd_symptoms_true == 0, True)
    return s

In [ ]:
testing_demand_models = {
    "symptoms": {
        "model": demand_test_if_experience_symptoms,
        # "loc": ...,
    }
}

### Allocation models

In [1]:
def allocate_tests(demands_test, states, params):
    """Allocate tests to individuals who demand a test.
    
    For simplicity, we assume that everyone who demands a test
    receives one.
    
    Args:
        demands_test (pandas.Series): A series with boolean entries
            where ``True`` indicates individuals asking for a test.
        states (pandas.DataFrame): The states of the individuals.
        params (pandas.DataFrame): A DataFrame with parameters.
    
    Returns:
        received_test (numpy.ndarray, pandas.Series): An array or a
            series which indicates which individuals received a test.
    
    """
    received_test = demands_test.copy()
    return received_test

In [ ]:
testing_allocation_models = {
    "direct_allocation": {
        "model": allocate_tests,
        # "loc": ...,
    }
}

### Processing models

In [2]:
def process_tests(pending_tests, states, params):
    """Process tests.
    
    For simplicity, we assume that all tests are processed immediately, without
    further delay.
    
    Args:
        pending_tests (pandas.Series): A series with boolean entries
            indicating whether the test of an individual is still waiting
            to be processed.
        states (pandas.DataFrame): The states of the individuals.
        params (pandas.DataFrame): A DataFrame with parameters.
    
    Returns:
        started_processing (numpy.ndarray, pandas.Series): An array or series
            with boolean entries indicating which tests started to be processed.    
    
    """
    started_processing = pending_tests.copy()
    return started_processing

In [ ]:
testing_processing_models = {
    "direct_processing": {
        "model": process_tests,
        # "loc": ...,
    }
}